In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.datasets import load_boston
from sklearn import datasets
from sklearn.model_selection import train_test_split
import math

# delta2 starts from 0 and delta1 starts from 0
def sigmoid(x,w):
  return 1/(1+ pow(math.e,-(np.dot(x,w))))


def predict(W1,W2,x,hiddenNeurons,hiddenBias):

  h = [random.uniform(-0.3,0.3) for i in range(hiddenNeurons)] # or hiddenNeurons

  d = [random.uniform(-0.3,0.3) for i in range(len(W2))]

  h[0] = hiddenBias

  # Input to hidden
  for i in range(1,hiddenNeurons):
    h[i] = sigmoid(x,W1[i-1]);

    # Hidden to Output
  for i in range(len(W2)):
    d[i] = sigmoid(h,W2[i])


  return d.index(max(d))


  




def findAccuracy(test_set,W1,W2,hiddenNeurons,hiddenBias):

  

  a = np.array(df.values)

  X = a[:,0:len(a[0])-1]

  Y = a[:,len(a[0])-1]

  confusionMatrix = np.zeros((len(classes),len(classes)),dtype='int')

    
  correctlyPredicted = 0

  for k in range(len(X)) :
    x = X[k]

    predictedClass = predict(W1,W2,x,hiddenNeurons,hiddenBias)

    if(predictedClass == Y[k]):
      correctlyPredicted += 1

    confusionMatrix[predictedClass][int(Y[k])] += 1

  
  print('Accuracy : ',correctlyPredicted*100/len(X))
  print('\n',confusionMatrix)
  for i in range(len(confusionMatrix)):
    print('precision for ',i,' class : ',confusionMatrix[i][i]/(sum(confusionMatrix[i])))

  print()

  colWiseSum_ConfusionMatrix = np.sum(confusionMatrix,axis=0)
  for i in range(len(colWiseSum_ConfusionMatrix)):
    print('recall for ',i,' class : ',confusionMatrix[i][i]/(colWiseSum_ConfusionMatrix[i]))

  
  print('\n\n')


  return correctlyPredicted*100/len(X)

    

def MLP(df,W1,W2,hiddenNeurons,alpha,rho,epochs,showPlot):

  a = np.array(df.values)

  X = a[:,0:len(a[0])-1]

  Y = a[:,len(a[0])-1]

  
  itr = 1

  epoch_graph = []
  mse_graph = []


  # one hot encoding
  y = []

  for i in range(len(Y)):
    yi = []
    for j in range(len(classes)):
      if(j==Y[i]):
        yi.append(1)
      else:
        yi.append(0)

    y.append(yi)



  prevJ = 0

  h = [random.uniform(-0.3,0.3) for i in range(hiddenNeurons)] # or hiddenNeurons

  d = [random.uniform(-0.3,0.3) for i in range(len(W2))]

  while itr <= epochs:

    a = np.array(df.values)

    X = a[:,0:len(a[0])-1]

    Y = a[:,len(a[0])-1]

    

    delta2 = np.ones(len(W2))
    delta1 = np.ones(len(W1))

    currJ = 0


    for k in range(len(X)) :
      x = X[k]

      # Input to hidden
      for i in range(1,hiddenNeurons):
        h[i] = sigmoid(x,W1[i-1]);

      # Hidden to Output
      for i in range(len(W2)):
        d[i] = sigmoid(h,W2[i]);

      # Local Gradient of Error in Each Output Node
      for i in range(len(W2)):
        delta2[i] = d[i] * (1-d[i]) * (y[k][i] - d[i]);

      # Local Gradient of Error in Each Hidden Node
      for i in range(len(W1)):
        delta1[i] = 0
        for j in range(len(W2)):
          delta1[i] += delta2[j] * W2[j][i]
        delta1[i] *= h[i] * (1 - h[i])

      # Updation of Weights - Hidden to Output
      for i in range(len(W2)):
        for j in range(len(W2[i])):
          W2[i][j] = W2[i][j] + alpha * delta2[i] * h[j];

      # Updation of Weights - Input to Hidden
      for i in range(len(W1)):
        for j in range(len(W1[i])):
          W1[i][j] = W1[i][j] + alpha * delta1[i] * x[j]

      for i in range(len(W2)):
        currJ += pow(y[k][i] - d[i],2)

      
    currJ /= 2*len(X)

    epoch_graph.append(itr)
    mse_graph.append(currJ)

    # print('prev : ',prevJ, 'curr : ',currJ)

    if(abs(currJ - prevJ) < rho):
      break

    prevJ = currJ

    itr += 1  

    
  if(showPlot != ''):

    plt.plot(epoch_graph,mse_graph,label=str(showPlot))
    plt.xlabel("EPOCHS")
    # naming the y axis
    plt.ylabel("MSE")
    plt.legend()
    plt.show()
  return W1,W2,h[0],currJ




      
      
















In [2]:
data = pd.DataFrame((datasets.load_wine()).data)
target = pd.DataFrame((datasets.load_wine()).target)


df = data;
df = df/(df.max().max())
df.insert(loc=len(df.columns),column = 'last',value=target)
df.insert(loc=0,column='-1',value=[1 for i in range(len(df))])

# shuffling DataFrame

df = df.sample(frac = 1)

x = 55
y = 15
z = 30
train_set, reamining_set = train_test_split(df,train_size = x/100)
validation_set, test_set = train_test_split(reamining_set,train_size = y/(y+z))

print('Train : ',len(train_set))
print('Valid : ',len(validation_set))
print("Test  : ",len(test_set))

alphaarr = np.array([0.1,0.001,0.1,0.03,0.15,0.1])
rhoarr = np.array([0.0001,0.01,0,0.001,0.0001,0])
epocharr = np.array([100,500,1000,400,500,1000])
hiddenNeuronsarr = np.array([4,36,30,3,5,20])

classes = np.unique(np.array(list(df['last'])))

# intialW = [random.uniform(-0.3,0.3) for i in range(len(df.columns)-1)]

# W1 = [intialW.copy() for i in range(len(classes)) ]

minHPIndex = 0
minJ = None
for i in range(len(alphaarr)):
  intialW = [random.uniform(-0.3,0.3) for j in range(len(df.columns)-1)]
  W1 = [intialW.copy() for j in range(hiddenNeuronsarr[i]-1)]
  intialW = [random.uniform(-0.3,0.3) for j in range(hiddenNeuronsarr[i])]
  W2 = [intialW.copy() for j in range(len(classes))]
  w1,w2,hiddenBias,j = MLP(validation_set,W1.copy(),W2.copy(),hiddenNeuronsarr[i],alphaarr[i],rhoarr[i],epocharr[i],'')
  if(minJ == None):
    minJ = j
  
  if(j < minJ):
    minHPIndex = i
    minJ = j

  print(j)
  # break

print('MinJ : ',minJ)
print('minHPIndex : ',minHPIndex)

intialW = [random.uniform(-0.3,0.3) for i in range(len(df.columns)-1)]
W1 = [intialW.copy() for i in range(hiddenNeuronsarr[minHPIndex]-1)]
intialW = [random.uniform(-0.3,0.3) for i in range(hiddenNeuronsarr[minHPIndex])]
W2 = [intialW.copy() for i in range(len(classes))]

# training
trainW1,trainW2,hiddenBias,J = MLP(train_set,W1.copy(),W2.copy(),hiddenNeuronsarr[minHPIndex],alphaarr[minHPIndex],rhoarr[minHPIndex],epocharr[minHPIndex],'')

#Testing
findAccuracy(test_set,trainW1,trainW2,hiddenNeuronsarr[minHPIndex],hiddenBias)

Train :  97
Valid :  27
Test  :  54
0.3335218101343339
0.35006468370969507
0.2220516081157398
0.35261325259240145
0.33432804321926646
0.21819002738795562
MinJ :  0.21819002738795562
minHPIndex :  5
Accuracy :  69.66292134831461

 [[55  4  6]
 [ 1 62 35]
 [ 3  5  7]]
precision for  0  class :  0.8461538461538461
precision for  1  class :  0.6326530612244898
precision for  2  class :  0.4666666666666667

recall for  0  class :  0.9322033898305084
recall for  1  class :  0.8732394366197183
recall for  2  class :  0.14583333333333334





69.66292134831461

In [3]:
from sklearn.model_selection import KFold

# https://stackoverflow.com/questions/19155718/select-pandas-rows-based-on-list-index
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html


print(df)

kf = KFold(n_splits=5)
kf

avgAccuracy = 0
foldNum = 1
for train_index, test_index in kf.split(np.array(df.values)):

  

  trainDf = df.iloc[train_index]
  testDf = df.iloc[test_index]

  print(train_index)
  


  print('FOLD : ',foldNum)

  # training
  trainW1,trainW2,hiddenBias,J = MLP(trainDf,W1.copy(),W2.copy(),hiddenNeuronsarr[minHPIndex],alphaarr[minHPIndex],rhoarr[minHPIndex],epocharr[minHPIndex],'')

  #Testing
  
  avgAccuracy += findAccuracy(testDf,trainW1,trainW2,hiddenNeuronsarr[minHPIndex],hiddenBias)

  print('--------------')


  foldNum += 1

  

avgAccuracy /= 5

print('AVG Accuracy : ',avgAccuracy)




     -1         0         1         2  ...        10        11        12  last
114   1  0.007190  0.000827  0.001488  ...  0.000554  0.001899  0.229167     1
148   1  0.007929  0.001929  0.001417  ...  0.000327  0.000964  0.386905     2
121   1  0.006881  0.001220  0.001923  ...  0.000554  0.002196  0.276786     1
112   1  0.007000  0.001595  0.001738  ...  0.000732  0.001488  0.361310     1
14    1  0.008560  0.001113  0.001417  ...  0.000714  0.001786  0.920833     0
..   ..       ...       ...       ...  ...       ...       ...       ...   ...
62    1  0.008137  0.000744  0.001143  ...  0.000732  0.001464  0.375000     1
48    1  0.008393  0.001202  0.001429  ...  0.000637  0.001637  0.630952     0
99    1  0.007315  0.001887  0.001315  ...  0.000845  0.001685  0.241667     1
17    1  0.008232  0.000935  0.001560  ...  0.000673  0.001530  0.672619     0
87    1  0.006935  0.000994  0.001560  ...  0.000810  0.001911  0.334524     1

[178 rows x 15 columns]
[ 36  37  38  39  40  41  4